In [7]:
from PIL import Image
import cv2
import numpy as np
import os

# Load image
img_path = r"E:\imgs\imgs\w_1.jpg"
img_pil = Image.open(img_path).convert("RGB")
img = np.array(img_pil)

H, W, _ = img.shape

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# Compute edges (texture proxy)
edges = cv2.Canny(gray, threshold1=50, threshold2=150)

# Divide image into a coarse grid (attention candidates)
grid_size = 3  # 3x3 grid
patch_h = H // grid_size
patch_w = W // grid_size

scores = []
patches = []

for i in range(grid_size):
    for j in range(grid_size):
        y1, y2 = i * patch_h, (i + 1) * patch_h
        x1, x2 = j * patch_w, (j + 1) * patch_w
        
        patch_edges = edges[y1:y2, x1:x2]
        score = patch_edges.mean()  # edge density
        
        scores.append(score)
        patches.append((x1, y1, x2, y2))

# Pick top-3 most "interesting" regions
top_idxs = np.argsort(scores)[-3:]

out_dir = "auto_candidates"
os.makedirs(out_dir, exist_ok=True)

for k, idx in enumerate(top_idxs):
    x1, y1, x2, y2 = patches[idx]
    crop = img_pil.crop((x1, y1, x2, y2))
    crop.save(os.path.join(out_dir, f"candidate_{k}.png"))

print("Saved 3 automatic candidate regions")


Saved 3 automatic candidate regions


In [8]:
# Simple evidence aggregation (manual for now)

# Pretend these came from a classifier later
# 1 = whale-like, 0 = background
candidate_labels = {
    "candidate_0": 1,  # head
    "candidate_1": 1,  # tail
    "candidate_2": 0,  # water
}

# AND rule: require at least 2 whale-like regions
whale_votes = sum(candidate_labels.values())

if whale_votes >= 2:
    decision = "WHALE PRESENT"
else:
    decision = "NO WHALE / UNCERTAIN"

print("Votes:", whale_votes)
print("System decision:", decision)


Votes: 2
System decision: WHALE PRESENT


In [ ]:
import cv2
import numpy as np
from PIL import Image

# Load ONE candidate patch (change index if needed)
patch_path = "auto_candidates/candidate_0.png"

patch = Image.open(patch_path).convert("RGB")
patch_np = np.array(patch)

# Convert to grayscale
gray = cv2.cvtColor(patch_np, cv2.COLOR_RGB2GRAY)

# Edge detection
edges = cv2.Canny(gray, 50, 150)

# Simple feature: edge density
edge_density = edges.mean()

print("Edge density:", edge_density)
